In [1]:
import treecorr
import pyccl as ccl
import numpy as np
cosmo = ccl.Cosmology(Omega_c = 0.22, 
                      h = 0.71,
                      Omega_b = 0.0448,
                      n_s = 0.963,
                      sigma8 = 0.801,
                      Omega_k = 0.0,
                      w0 = -1.00,
                      #Omega_nu = 0.0
                      wa = 0.0,)

We load all of the data into a single array for simplicity, n=6 is used for consistency so that we can start counting from 1 to 5 instead of from 0 to 4

In [2]:
data = np.empty((6, 1113829, 10))
for i in np.arange(5)+1:
    data[i] = np.loadtxt('/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(i))      
        

Here we calculate the auto correlation functions for all bins

In [ ]:
for i in np.arange(5)+1:
    ra = data[i,:,0]
    dec = data[i,:,1]
    g1 = data[i,:,2]
    g2 = data[i,:,3]
    w = data[i,:,4]
    z = data[i,:,5]
    r = ccl.comoving_radial_distance(cosmo, 1./(1+z))
    #cat = treecorr.Catalog(ra = ra, dec=dec, g1=g1, g2=g2, w=w, r=r, ra_units='degrees', dec_units='degrees')
    cat = treecorr.Catalog(ra = ra, dec=dec, g1=g1, g2=g2, w=w, r=r, ra_units='degrees', dec_units='degrees',flip_g2='True')

    gg = treecorr.GGCorrelation(nbins = 9,
                       min_sep = 0.00833333, #0.5 arcmin
                       max_sep = 7.925, #475.5 arcmin
                       bin_slop = 0.01) 
    #gg.process(cat, metric = 'Rperp')
    gg.process(cat, metric = 'Arc')

    gg.write('correlations/cross_correlation_tomo{}_{}'.format(i,i))

Here we calculate the cross correlation functions for all bins

In [6]:
for i in np.arange(5)+1:
    for j in range(1, i):
        ra_1 = data[i,:,0]
        dec_1 = data[i,:,1]
        g1_1 = data[i,:,2]
        g2_1 = data[i,:,3]
        w_1 = data[i,:,4]
        z_1 = data[i,:,5]
        r_1 = ccl.comoving_radial_distance(cosmo, 1./(1+z_1))
        cat_1 = treecorr.Catalog(ra = ra_1, dec=dec_1, g1=g1_1, g2=g2_1, w=w_1, r=r_1, ra_units='degrees', dec_units='degrees')
        ra_2 = data[j,:,0]
        dec_2 = data[j,:,1]
        g1_2 = data[j,:,2]
        g2_2 = data[j,:,3]
        w_2 = data[j,:,4]
        z_2 = data[j,:,5]
        r_2 = ccl.comoving_radial_distance(cosmo, 1./(1+z_2))
        cat_2 = treecorr.Catalog(ra = ra_2, dec=dec_2, g1=g1_2, g2=g2_2, w=w_2, r=r_2, ra_units='degrees', dec_units='degrees')
        gg = treecorr.GGCorrelation(nbins = 9,
                       min_sep = 0.00833333, #0.5 arcmin
                       max_sep = 7.925, #475.5 arcmin
                       bin_slop = 0.01) 
        gg.process(cat_1, cat_2, metric = 'Rperp')
        gg.write('correlations/cross_correlation_tomo_{}_{}'.format(i,j))

Here we show how to read the correlation functions saved to a file by TreeCorr

gg = treecorr.GGCorrelation(nbins = 9,
                       min_sep = 0.00833333, #0.5 arcmin
                       max_sep = 7.925, #475.5 arcmin
                       bin_slop = 0.01) 
gg.read('correlations/auto_correlation_tomo3')

In [25]:
gg.xip

array([2.3351e-04, 2.2439e-04, 2.2668e-04, 2.1037e-04, 2.0577e-04,
       1.6077e-04, 9.8708e-05, 5.3405e-05, 2.7840e-05])